In [ ]:
from summer2.functions import time as stf
import pandas as pd
import numpy as np
from tbdynamics.tools.detect import get_interpolation_rates_from_annual
import math
from typing import List, Dict, Any

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
act_trial_reported_rates = {
    2014.9: 0.0,
    2015: 1.85,
    2016: 1.61,
    2017: 1.51,
    2018: 1.36,
    2018.1: 0.00,
}

In [ ]:
# act_trial_screening_rate = {
#         2014.0: 0.0,
#         2015.0: 1.83,
#         2015.1: 1.66,
#         2016.0: 1.66,
#         2016.1: 1.51,
#         2017.0: 1.51,
#         2017.1: 1.35,
#         2018.0: 1.35,
#         2018.1: 0.00,
#     }
act_trial_screening_rate = get_interpolation_rates_from_annual(act_trial_reported_rates)

In [ ]:
f_sig = stf.get_linear_interpolation_function(list(act_trial_screening_rate.keys()), list(act_trial_screening_rate.values()))
trial_year = np.linspace(2013,2020.5,10000)
pd.Series(stf.get_time_callable(f_sig)(trial_year), index=trial_year).plot()

In [ ]:
act_control_screening_rate = {
        2017.9: 0.0,
        2018: 1.14,
        2018.1: 0.0,
}
act_control_screening_rate = get_interpolation_rates_from_annual(act_control_screening_rate)

In [ ]:
f_sig = stf.get_linear_interpolation_function(list(act_control_screening_rate.keys()), list(act_control_screening_rate.values()))
control_year = np.linspace(2016,2020.5,100000)
pd.Series(stf.get_time_callable(f_sig)(control_year), index=control_year).plot()

In [ ]:
def make_future_acf_scenarios(config: Dict[str, List]) -> Dict[str, Dict[float, float]]:
    """
    Generates future ACF screening rate scenarios based on frequency and coverage.
    
    Args:
        config (Dict): Configuration dictionary containing arm, frequency, and coverage settings.
        
    Returns:
        Dict: A dictionary where the keys are scenario labels and values are rate dictionaries.
    """
    scenarios = {}
    arms = config.get("arm", [])
    every = config.get("every", [])
    coverages = config.get("coverage", [])

    for arm in arms:
        for freq in every:
            assert freq in [2, 4], f"Frequency must be 2 or 4, got {freq}"
            years = list(range(2027, 2036, freq))  # Screening years
            for cov in coverages:
                assert 0 < cov <= 1.0, f"Coverage must be in (0,1], got {cov}"
                rate = -math.log(1 - cov)
                scenario_key = f"{arm}_{freq}_{int(cov * 100)}"
                rate_dict = {2026.0: 0.0}
                rate_dict.update({float(year): rate for year in years})
                rate_dict[2035.1] = 0.0
                scenarios[scenario_key] = rate_dict
    return scenarios

In [ ]:
config = {
    "arm": ["trial", "control"],
    "every": [2],
    "coverage": [0.5,0.8],
}
future_acf_scenario = make_future_acf_scenarios(config)